This notebook is where I did the bulk of the operations for cleaning the data. It follows the steps outlined in the slides for how I cleaned this data.

In [ ]:
import pandas as pd
from datetime import datetime, date
import re
from functions import convertTime, timeDifference, timeOfDay

In [ ]:
df = pd.read_csv('data.csv') # Resets the dataframe | FOR TESTING

In [ ]:
print(df.to_string()) # Displays dataframe | FOR TESTING

The next cell removes any rows that are missing data in at least 2 of CHECKIN_DTTM, CHAIR_START, INFUSION_START.  
This also cleans up any appointments that were cancelled or skipped.  

In [ ]:
df = df.dropna(subset=['CHECKIN_DTTM', 'CHAIR_START', 'INFUSION_START'], thresh=2)

The next cell applies the convertTime function to these columns, converting the times within the columns to datetime objects, allowing for easier addition and subtraction of them later.  
convertTime can be found in functions.py 

In [ ]:
df['CHECKIN_DTTM'] = df['CHECKIN_DTTM'].apply(convertTime)
df['CHAIR_START'] = df['CHAIR_START'].apply(convertTime)
df['INFUSION_START'] = df['INFUSION_START'].apply(convertTime)

The next cell is reponsible for dropping all except one of the duplicates.  
First it sorts the data by 'INFUSION_START', then 'INPATIENT_DATA_ID_x', which serves to sort all the duplicates in place by their infusion start time.  
This follows the assumption that the patient's wait ends when their first infusion begins.
Then, every duplicate row except the first (earliest infusion start), is dropped. 

In [ ]:
df = df.sort_values(by=['INFUSION_START'])
df = df.sort_values(by=['INPATIENT_DATA_ID_x'])

df.drop_duplicates(subset='INPATIENT_DATA_ID_x', keep='first', inplace=True)

The next cell calculates the wait time between CHECKIN_DTTM and CHAIR_START (WAIT_ONE). This marks the first time the patients are waiting.  
Next, it calculates the wait time between CHAIR_START AND INFUSION_START (WAIT_TWO). This marks the second time the patients are waiting.  
timeDifference is a function I created to calculate the difference in times. It can be found in functions.py

In [ ]:
df['WAIT_ONE'] = df.apply(lambda row: timeDifference(row['CHECKIN_DTTM'], row['CHAIR_START']), axis=1)
df['WAIT_TWO'] = df.apply(lambda row: timeDifference(row['CHAIR_START'], row['INFUSION_START']), axis=1)

This next cell filters the wait time columns. If the wait time is negative, then that row must have invalid data, so I kept only the rows with valid time data.  
Also added a total wait time column that added both columns up.

In [ ]:
df = df[(df['WAIT_ONE'] >= 0) | (df['WAIT_ONE'].isna())]
df = df[(df['WAIT_TWO'] >= 0) | (df['WAIT_TWO'].isna())]
df['TOTAL_WAIT'] = df['WAIT_ONE'] + df['WAIT_TWO']

This next cell adds the 'TIME_OF_DAY' column to the dataset.  
This is for use in the Time of Day vs. Average Wait Time graph.  
timeOfDay can be found in functions.py

In [ ]:
df['APPT_DTTM'] = pd.to_datetime(df['APPT_DTTM'])
df['TIME_OF_DAY'] = df['APPT_DTTM'].apply(timeOfDay)

This cell exports the dataframe into a CSV, ready for statistical analysis.

In [ ]:
df.to_csv('finalData.csv', index=False)